## NTUEE儀表板資料處理中心 - 祝您好孕
##### 作者：NTUEE儀表板資料組

#### 引入必要套件與函數

In [1]:
%%capture 
!pip3 install pandas
!pip3 install geocoder
!pip3 install geopandas
!pip3 install shapely
!pip3 install openpyxl

##### 安裝套件、函數宣告


In [2]:
%%capture
import requests
import pandas as pd
import geocoder
import geopandas as gpd
from shapely.geometry import Point, Polygon
from IPython.display import display, HTML
## 從data.taipei抓取資料
def get_datataipei_api(rid):
    url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire"""
    response = requests.request("GET", url)
    data_dict = response.json()
    count = data_dict['result']['count']
    res = list()
    offset_count = int(count/1000)
    for i in range(offset_count+1):
        i = i* 1000
        url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire&offset={i}&limit=1000"""
        response = requests.request("GET", url)
        get_json = response.json()
        res.extend(get_json['result']['results'])
    return pd.DataFrame(res)
## 用geocoder取得TWD97座標
def 新增經緯度TWD97座標行以及轉GeoDataFrame(資料, 地址欄位名稱):
	資料['geometry'] = 資料[地址欄位名稱].apply(
		lambda x: (
			lambda a: Point(a.json['lng'], a.json['lat']) if a and a.json else None
		)(geocoder.arcgis(x))
	)
	# 沒查到經緯度的話...郭囉 誰叫你選那麼奇怪的址
	資料.dropna(subset=['geometry'], inplace=True)
	資料 = gpd.GeoDataFrame(
		資料,
		geometry='geometry', 
		crs='EPSG:4326'
	)
	# 資料 = 資料.to_crs('EPSG:3826') # TWD97
	return 資料
## 印資料
def 印資料(資料):
	print("這份資料總共有", 資料.shape[0], "筆！")
	display(資料)

### 祝您好孕

##### 資料庫（不斷更新！）


In [14]:
祝您好孕資料庫 = {
    "112上半年婦幼安全警示地點": "016bf772-71bd-4500-aba2-22387a1d0ae7",
	"公辦幼兒園班級數": "./公辦幼兒園.xlsx",
	"哺集乳室": "4350117f-8697-4903-8a49-a7a59f30eeb1"
}

##### 點子
1. 婦幼安全警示區分佈圖
2. 幼兒園分佈圖

##### 婦幼安全警示區分佈圖

In [4]:
# 取得資料
婦幼安全警示區資料 = get_datataipei_api(祝您好孕資料庫["112上半年婦幼安全警示地點"])
婦幼安全警示區資料.drop('_id', axis=1, inplace=True)
婦幼安全警示區資料.drop('_importdate', axis=1, inplace=True)
印資料(婦幼安全警示區資料)


這份資料總共有 20 筆！


,編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註
0,1\t臺北市大同區捷運中山站\t大同分局\t103\t續列
1,2\t臺北市萬華區峨嵋街、漢中街口(捷運西門站)\t萬華分局\t108\t續列
2,3\t臺北市萬華區西園路、和平西路口(捷運龍山寺站)\t萬華分局\t108\t續列
3,4\t臺北市中山區捷運松江南京站\t中山分局\t104\t續列
4,5\t臺北市中山區捷運中山國小站\t中山分局\t104\t續列
5,6\t臺北市大安區捷運忠孝復興站\t大安分局\t106\t續列
6,7\t臺北市中正區捷運臺北車站\t中正第一分局\t100\t續列
7,8\t臺北市中正區二二八公園\t中正第一分局\t100\t續列
8,9\t臺北市中正區捷運古亭站\t中正第二分局\t100\t續列
9,10\t臺北市松山區新東街9巷\t松山分局\t105\t續列


In [5]:
# 整理一下資料
婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'][10] += 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'][11]
婦幼安全警示區資料.drop(11, inplace=True)
婦幼安全警示區資料.reset_index(drop=True, inplace=True)
婦幼安全警示區資料['地點位置'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[1]
婦幼安全警示區資料['管轄分局'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[2]
婦幼安全警示區資料.drop('編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註', axis=1, inplace=True)
印資料(婦幼安全警示區資料)

這份資料總共有 19 筆！


,地點位置,管轄分局
0,臺北市大同區捷運中山站,大同分局
1,臺北市萬華區峨嵋街、漢中街口(捷運西門站),萬華分局
2,臺北市萬華區西園路、和平西路口(捷運龍山寺站),萬華分局
3,臺北市中山區捷運松江南京站,中山分局
4,臺北市中山區捷運中山國小站,中山分局
5,臺北市大安區捷運忠孝復興站,大安分局
6,臺北市中正區捷運臺北車站,中正第一分局
7,臺北市中正區二二八公園,中正第一分局
8,臺北市中正區捷運古亭站,中正第二分局
9,臺北市松山區新東街9巷,松山分局


In [6]:
# 找座標，並轉成GeoDataFrame
婦幼安全警示區資料GDF = 新增經緯度TWD97座標行以及轉GeoDataFrame(婦幼安全警示區資料, '地點位置')
印資料(婦幼安全警示區資料GDF)

這份資料總共有 18 筆！


,地點位置,管轄分局,geometry
0,臺北市大同區捷運中山站,大同分局,POINT (121.51541 25.06593)
1,臺北市萬華區峨嵋街、漢中街口(捷運西門站),萬華分局,POINT (121.50696 25.04150)
2,臺北市萬華區西園路、和平西路口(捷運龍山寺站),萬華分局,POINT (121.49413 25.03575)
3,臺北市中山區捷運松江南京站,中山分局,POINT (121.53331 25.06443)
4,臺北市中山區捷運中山國小站,中山分局,POINT (121.52683 25.06280)
5,臺北市大安區捷運忠孝復興站,大安分局,POINT (121.54304 25.04157)
6,臺北市中正區捷運臺北車站,中正第一分局,POINT (121.51579 25.04743)
7,臺北市中正區二二八公園,中正第一分局,POINT (121.51856 25.03247)
8,臺北市中正區捷運古亭站,中正第二分局,POINT (121.51856 25.03247)
9,臺北市松山區新東街9巷,松山分局,POINT (121.56510 25.05785)


In [7]:
# 儲存成GeoJSON
婦幼安全警示區資料GDF.to_file(
    './112上半年婦幼安全警示區地點.geojson',
    driver='GeoJSON'
)

##### 幼兒園分佈圖

In [8]:
# 取得資料
公辦幼兒園班級數 = pd.read_excel(祝您好孕資料庫["公辦幼兒園班級數"])
印資料(公辦幼兒園班級數)

這份資料總共有 147 筆！


,區,校園名稱,3-5\n歲班,2 歲專班,總班\n數,3-5歲班缺額,2 歲專班缺額,總缺\n額
0,松山區,松山附幼,5,2.0,7,13,31.0,44
1,松山區,**西松附幼,7,2.0,9,49,30.0,79
2,松山區,**敦化附幼,4,NaN,4,64,NaN,64
3,松山區,民權附幼,6,2.0,8,47,32.0,79
4,松山區,民族附幼,6,NaN,6,61,NaN,61
...,...,...,...,...,...,...,...,...
142,北投區,義方附幼,3,1.0,4,3,1.0,4
143,北投區,立農附幼,4,NaN,4,4,NaN,4
144,北投區,明德附幼,4,2.0,6,4,2.0,6
145,北投區,文化附幼,2,NaN,2,2,NaN,2


In [9]:
# 整理一下資料
公辦幼兒園班級數.fillna(0, inplace=True)
公辦幼兒園班級數.columns = 公辦幼兒園班級數.columns.str.replace('\n', '')
公辦幼兒園班級數.columns = 公辦幼兒園班級數.columns.str.replace(' ', '')
公辦幼兒園班級數['校園名稱'] = 公辦幼兒園班級數['校園名稱'].str.replace('*', '')
公辦幼兒園班級數['2歲專班'] = 公辦幼兒園班級數['2歲專班'].astype(int)
公辦幼兒園班級數['2歲專班缺額'] = 公辦幼兒園班級數['2歲專班缺額'].astype(int)
公辦幼兒園班級數['核定人數'] = 公辦幼兒園班級數['總班數'].astype(int) * 30
公辦幼兒園班級數['實際就讀人數'] = 公辦幼兒園班級數['核定人數'] - 公辦幼兒園班級數['總缺額']
公辦幼兒園班級數['校園名稱'] = 公辦幼兒園班級數['區'] + 公辦幼兒園班級數['校園名稱']
公辦幼兒園班級數.drop('區', axis=1, inplace=True)
印資料(公辦幼兒園班級數)

這份資料總共有 147 筆！


,校園名稱,3-5歲班,2歲專班,總班數,3-5歲班缺額,2歲專班缺額,總缺額,核定人數,實際就讀人數
0,松山區松山附幼,5,2,7,13,31,44,210,166
1,松山區西松附幼,7,2,9,49,30,79,270,191
2,松山區敦化附幼,4,0,4,64,0,64,120,56
3,松山區民權附幼,6,2,8,47,32,79,240,161
4,松山區民族附幼,6,0,6,61,0,61,180,119
...,...,...,...,...,...,...,...,...,...
142,北投區義方附幼,3,1,4,3,1,4,120,116
143,北投區立農附幼,4,0,4,4,0,4,120,116
144,北投區明德附幼,4,2,6,4,2,6,180,174
145,北投區文化附幼,2,0,2,2,0,2,60,58


In [10]:
# 找座標，並轉成GeoDataFrame
公辦幼兒園班級數GDF = 新增經緯度TWD97座標行以及轉GeoDataFrame(公辦幼兒園班級數, '校園名稱')
印資料(公辦幼兒園班級數GDF)

這份資料總共有 146 筆！


,校園名稱,3-5歲班,2歲專班,總班數,3-5歲班缺額,2歲專班缺額,總缺額,核定人數,實際就讀人數,geometry
0,松山區松山附幼,5,2,7,13,31,44,210,166,POINT (118.60802 42.40046)
1,松山區西松附幼,7,2,9,49,30,79,270,191,POINT (121.57729 25.05003)
2,松山區敦化附幼,4,0,4,64,0,64,120,56,POINT (121.57729 25.05003)
3,松山區民權附幼,6,2,8,47,32,79,240,161,POINT (121.57729 25.05003)
4,松山區民族附幼,6,0,6,61,0,61,180,119,POINT (121.57729 25.05003)
...,...,...,...,...,...,...,...,...,...,...
142,北投區義方附幼,3,1,4,3,1,4,120,116,POINT (121.50253 25.13289)
143,北投區立農附幼,4,0,4,4,0,4,120,116,POINT (121.50359 25.12021)
144,北投區明德附幼,4,2,6,4,2,6,180,174,POINT (121.50253 25.13289)
145,北投區文化附幼,2,0,2,2,0,2,60,58,POINT (121.50253 25.13289)


In [11]:
公辦幼兒園班級數GDF.to_file(
    './112學年度公辦幼兒園班級數.geojson',
    driver='GeoJSON'
)

##### 哺集乳室

In [185]:
# 取得資料
哺集乳室資料 = get_datataipei_api(祝您好孕資料庫["哺集乳室"])
哺集乳室資料.drop('_id', axis=1, inplace=True)
哺集乳室資料.drop('_importdate', axis=1, inplace=True)
印資料(哺集乳室資料)

這份資料總共有 538 筆！


,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,優良哺集乳室認證效期,輪椅使用,貼心小提醒
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,無,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,112年8月,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,無,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主


In [215]:
# Custom function to process each row
哺集乳室資料深拷貝 = 哺集乳室資料.copy(deep=True)
def 處理基本資料字串(列, 獨特元素列表):
    if "以室內設備為主" in 列 or "以上皆無提供" in 列:
        return "無"
    設備字典 = {}
    for 元素 in 獨特元素列表:
        元素 = 元素.strip()
        if 元素 and 元素 != "以室內設備為主" and 元素 != "以上皆無提供" and 元素 != "\u3000music" and 元素 != "music":
            設備字典[元素] = "有" if 元素 in 列 else "無"
        if 元素 == '毒器具':
            設備字典['消毒器具'] = "有"
            設備字典.pop(元素)
        if 元素 == '母乳專用冰箱消毒器具':
            設備字典['消毒器具'] = "有"
            設備字典['母乳專用冰箱'] = "有"
            設備字典.pop(元素)
        if 元素 in {'設置意見回饋', '設置滿意度調查表', '飲水服務在1樓大廳，有意見調查表 （回饋箱置於1樓大廳）', '箱進行滿意度調查', '滿意度調查', '進行滿意度調查', '設置意見回饋箱'}:
            設備字典['設置意見回饋或滿意度調查'] = "有"
            設備字典.pop(元素)
        if 元素 == '其他（多功能溫奶器）':
            設備字典['多功能溫奶器'] = "有"
            設備字典.pop(元素)
        if 元素 == '消毒器具置物空間':
            設備字典['消毒器具'] = "有"
            設備字典['置物空間'] = "有"
            設備字典.pop(元素)
        if 元素 == '母乳哺育宣導資訊箱':
            設備字典.pop(元素)
    return 設備字典
全部的元素 = set(';'.join(哺集乳室資料['友善設備或服務']).split(';'))
基本設備字典 = {
    '靠背椅': '有',
    '有蓋垃圾桶': '有',
    '電源設備 （插座或延長線）': '有',
    '可由內部上鎖之門': '有',
    '緊急求救設備 （求救鈴或電話等）': '有',
    '洗手設施 （洗手台或乾洗手液）': '有'
}
哺集乳室資料深拷貝['友善設備或服務'] = 哺集乳室資料深拷貝['友善設備或服務'].apply(lambda x: 處理基本資料字串(x.split(';'), 全部的元素))
哺集乳室資料深拷貝['基本設備'] = 哺集乳室資料深拷貝['基本設備'].apply(lambda x: 基本設備字典)
哺集乳室資料深拷貝['可提供輪椅媽媽使用'] = 哺集乳室資料深拷貝['輪椅使用'].apply(lambda x: "是" if x == "可提供輪椅媽媽使用" else "否")
哺集乳室資料深拷貝.drop('貼心小提醒', axis=1, inplace=True)
哺集乳室資料深拷貝.drop('優良哺集乳室認證效期', axis=1, inplace=True)
哺集乳室資料深拷貝.drop('輪椅使用', axis=1, inplace=True)



In [216]:
哺集乳室資料深拷貝

,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,可提供輪椅媽媽使用
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '無', '飲水服務': '有', '設置意見回饋或滿意度調查': '...",是
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '有', '飲水服務': '有', '設置意見回饋或滿意度調查': '...",否
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '無', '飲水服務': '有', '設置意見回饋或滿意度調查': '...",是
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '無', '飲水服務': '無', '設置意見回饋或滿意度調查': '...",是
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '無', '飲水服務': '有', '設置意見回饋或滿意度調查': '...",否
...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '無', '飲水服務': '有', '設置意見回饋或滿意度調查': '...",是
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...",無,否
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...","{'免費衛生用品': '有', '飲水服務': '有', '設置意見回饋或滿意度調查': '...",是
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,"{'靠背椅': '有', '有蓋垃圾桶': '有', '電源設備 （插座或延長線）': '有...",無,否
